In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import json
from typing import List, Dict, Any

In [ ]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device==0 else 'CPU'}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto" if device == 0 else None,
    torch_dtype=torch.bfloat16 if device == 0 else torch.float32,
    low_cpu_mem_usage=True,
)

# Llama-style padding: set pad to eos
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

chat_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=50,
)


In [ ]:
SYSTEM_PROMPT = "You are a careful academic assistant. Be precise and give clear structured output (not JSON, not CSV, no files)."

# --- Strict Prompt Functions ---
def build_detection_prompt(submission: str, few_shots: List[Dict[str, Any]]) -> List[Dict[str, str]]:
    """
    Academic Integrity Detector Prompt
    ----------------------------------
    Purpose:
        Classifies student submissions as Human, AI, or Hybrid (AI-assisted).

    Technique:
        - Role-based prompting
        - Few-shot support
        - CoT (reasoning encouraged but hidden from output)
        - Output in plain text

    Expected Output (example format in plain text):
        Label: Human | AI | Hybrid
        Rationale:
        - short bullet point 1
        - short bullet point 2
        Flags: style_inconsistency / high_verbatim / generic_phrasing / none
    """
    shot_texts = []
    for s in few_shots:
        shot_texts.append(
            f'Submission: """{s.get("final_submission","")}"""\n'
            f'Your analysis (2–4 bullet points): <analysis>\n'
            f'Label: {s.get("label_type","")}\n'
        )
    examples_block = "\n\n".join(shot_texts) if shot_texts else "/* no examples available */"
    user = f"""
You are an AI text-source classifier for academic integrity.
Decide whether the student submission is Human, AI, or Hybrid (AI-assisted).
Guidelines:
- Consider discourse features (specificity, subjectivity, personal context), style consistency, local/global coherence, repetitiveness, and cliché patterns.
- Hybrid = meaningful human writing with some AI assistance, or explicit admission of mixed use.
Examples:
{examples_block}
Now analyze the NEW submission and respond in plain text with the following structure:
Label: ...
Rationale:
- point 1
- point 2
Flags: ...
NEW submission:
\"\"\"{submission}\"\"\"\n
"""
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

In [ ]:
def build_feedback_prompt(domain: str, assignment_prompt: str, rubric_text: str, submission: str) -> List[Dict[str, str]]:
    """
    Rubric-Aligned Feedback Prompt
    ------------------------------
    Purpose:
        Generates structured, supportive feedback for a student submission.
    """
    user = f"""
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return plain structured text only (no JSON, no files).
Sections to include:
1) Overall Summary: 2–4 sentences on strengths and priorities.
2) Criteria Feedback: for each rubric criterion include:
   - Criterion
   - Rating (excellent, good, average, needs_improvement, poor)
   - Evidence (1–3 bullet points citing excerpts or behaviors)
   - Improvement Tip (one concrete step)
3) Suggested Grade: short string (optional)
Context:
- Domain: {domain}
- Assignment prompt: {assignment_prompt}
Rubric (verbatim):
{rubric_text}
Student submission:
\"\"\"{submission}\"\"\"\n
"""
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

In [ ]:
# --- Rubric formatting helper ---
def format_rubric(rubric):
    formatted_rubric = f"Rubric ID: {rubric['rubric_id']}\n\nCriteria:\n"
    for item in rubric['criteria']:
        formatted_rubric += f"Criterion: {item['criterion_id']}\nName: {item['name']}\nDescription: {item['description']}\nPerformance Descriptors:\n"
        for key, val in item['performance_descriptors'].items():
            formatted_rubric += f"  - {key}: {val}\n"
    return formatted_rubric

In [ ]:
# --- Load JSON data ---
with open("psychology.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
few_shots = data.get("few_shots", [])

# --- Loop through all submissions ---
for i, submission in enumerate(data['submissions'], 1):
    submission_text = submission['final_submission']
    label_type = submission.get("label_type", "Unknown")

    # Generate feedback
    feedback_messages = build_feedback_prompt(
        domain=data['domain'],
        assignment_prompt=data.get("prompt", "Analyze student submission and provide detailed feedback"),
        rubric_text=rubric_text,
        submission=submission_text
    )
    feedback_outputs = chat_pipe(
        feedback_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    feedback_response = feedback_outputs[0]['generated_text'].strip()

    # Run AI Detection
    detection_messages = build_detection_prompt(submission_text, few_shots)
    detection_outputs = chat_pipe(
        detection_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    detection_response = detection_outputs[0]['generated_text'].strip()

    # Print results
    print(f"\n--- SUBMISSION {i} (Label: {label_type}) ---")
    print("\n--- RUBRIC-ALIGNED FEEDBACK ---\n")
    print(feedback_response)
    print("\n--- ACADEMIC INTEGRITY DETECTION ---\n")
    print(detection_response)


In [ ]:
# --- Load JSON data ---
with open("accounting.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
few_shots = data.get("few_shots", [])

# --- Loop through all submissions ---
for i, submission in enumerate(data['submissions'], 1):
    submission_text = submission['final_submission']
    label_type = submission.get("label_type", "Unknown")

    # Generate feedback
    feedback_messages = build_feedback_prompt(
        domain=data['domain'],
        assignment_prompt=data.get("prompt", "Analyze student submission and provide detailed feedback"),
        rubric_text=rubric_text,
        submission=submission_text
    )
    feedback_outputs = chat_pipe(
        feedback_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    feedback_response = feedback_outputs[0]['generated_text'].strip()

    # Run AI Detection
    detection_messages = build_detection_prompt(submission_text, few_shots)
    detection_outputs = chat_pipe(
        detection_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    detection_response = detection_outputs[0]['generated_text'].strip()

    # Print results
    print(f"\n--- SUBMISSION {i} (Label: {label_type}) ---")
    print("\n--- RUBRIC-ALIGNED FEEDBACK ---\n")
    print(feedback_response)
    print("\n--- ACADEMIC INTEGRITY DETECTION ---\n")
    print(detection_response)


In [ ]:
# --- Load JSON data ---
with open("teaching.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
few_shots = data.get("few_shots", [])

# --- Loop through all submissions ---
for i, submission in enumerate(data['submissions'], 1):
    submission_text = submission['final_submission']
    label_type = submission.get("label_type", "Unknown")

    # Generate feedback
    feedback_messages = build_feedback_prompt(
        domain=data['domain'],
        assignment_prompt=data.get("prompt", "Analyze student submission and provide detailed feedback"),
        rubric_text=rubric_text,
        submission=submission_text
    )
    feedback_outputs = chat_pipe(
        feedback_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    feedback_response = feedback_outputs[0]['generated_text'].strip()

    # Run AI Detection
    detection_messages = build_detection_prompt(submission_text, few_shots)
    detection_outputs = chat_pipe(
        detection_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    detection_response = detection_outputs[0]['generated_text'].strip()

    # Print results
    print(f"\n--- SUBMISSION {i} (Label: {label_type}) ---")
    print("\n--- RUBRIC-ALIGNED FEEDBACK ---\n")
    print(feedback_response)
    print("\n--- ACADEMIC INTEGRITY DETECTION ---\n")
    print(detection_response)


In [ ]:
# --- Load JSON data ---
with open("engineering.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
few_shots = data.get("few_shots", [])

# --- Loop through all submissions ---
for i, submission in enumerate(data['submissions'], 1):
    submission_text = submission['final_submission']
    label_type = submission.get("label_type", "Unknown")

    # Generate feedback
    feedback_messages = build_feedback_prompt(
        domain=data['domain'],
        assignment_prompt=data.get("prompt", "Analyze student submission and provide detailed feedback"),
        rubric_text=rubric_text,
        submission=submission_text
    )
    feedback_outputs = chat_pipe(
        feedback_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    feedback_response = feedback_outputs[0]['generated_text'].strip()

    # Run AI Detection
    detection_messages = build_detection_prompt(submission_text, few_shots)
    detection_outputs = chat_pipe(
        detection_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    detection_response = detection_outputs[0]['generated_text'].strip()

    # Print results
    print(f"\n--- SUBMISSION {i} (Label: {label_type}) ---")
    print("\n--- RUBRIC-ALIGNED FEEDBACK ---\n")
    print(feedback_response)
    print("\n--- ACADEMIC INTEGRITY DETECTION ---\n")
    print(detection_response)


In [ ]:
# --- Load JSON data ---
with open("it.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
few_shots = data.get("few_shots", [])

# --- Loop through all submissions ---
for i, submission in enumerate(data['submissions'], 1):
    submission_text = submission['final_submission']
    label_type = submission.get("label_type", "Unknown")

    # Generate feedback
    feedback_messages = build_feedback_prompt(
        domain=data['domain'],
        assignment_prompt=data.get("prompt", "Analyze student submission and provide detailed feedback"),
        rubric_text=rubric_text,
        submission=submission_text
    )
    feedback_outputs = chat_pipe(
        feedback_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    feedback_response = feedback_outputs[0]['generated_text'].strip()

    # Run AI Detection
    detection_messages = build_detection_prompt(submission_text, few_shots)
    detection_outputs = chat_pipe(
        detection_messages,
        return_full_text=False,
        eos_token_id=tokenizer.eos_token_id
    )
    detection_response = detection_outputs[0]['generated_text'].strip()

    # Print results
    print(f"\n--- SUBMISSION {i} (Label: {label_type}) ---")
    print("\n--- RUBRIC-ALIGNED FEEDBACK ---\n")
    print(feedback_response)
    print("\n--- ACADEMIC INTEGRITY DETECTION ---\n")
    print(detection_response)
